# Coronavirus Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1636170550178_0014'

In [3]:
pd.options.display.max_columns = 35

### Create Variables for Queries

In [4]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [5]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
       
""")
stdValue = stdVar.first()['std_length_of_stay']
print(stdValue)

12.8


In [6]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
 
      and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
             or v.vaccine_date is null) 
       
""")
stdValue_nonVac = stdVar.first()['std_length_of_stay']
print(stdValue_nonVac)

12.81


In [7]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
               
      and  ( datediff(h.serviceDate, v.vaccine_date) > 0
             and v.vaccine_date is not null)        
      
    
""")
stdValue_vac = stdVar.first()['std_length_of_stay']
print(stdValue_vac)

5.62


In [8]:
spark.sql(f"""

          select
              case
                when age >= 90 then "90+"
                when age >= 85 and age < 90 then "85-89"
                when age >= 80 and age < 85 then "80-85"
                when age >= 75 and age < 80 then "75-79"
                when age >= 70 and age < 75 then "70-74"
                when age >= 65 and age < 70 then "65-69"
                when age >= 60 and age < 65 then "60-65"
                when age >= 55 and age < 60 then "55-59"
                when age >= 50 and age < 55 then "50-54"
                when age >= 45 and age < 50 then "45-49"
                when age >= 40 and age < 45 then "40-45"
                when age >= 35 and age < 40 then "35-39"
                when age >= 30 and age < 35 then "30-34"
                when age >= 25 and age < 30 then "25-29"
                when age >= 20 and age < 25 then "20-25"
                when age >= 12 and age < 20 then "12-19"
                when age >= 0 and age < 12 then "0-12"
              else "unknown"
              end as age_range
              , count(h.personId) as count_of_non_vaccinated_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_nonVac}

              and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
                     or v.vaccine_date is null) 
               
        group by 1
        order by 1
     
""").toPandas()

,age_range,count_of_non_vaccinated_patients,mean_length_of_stay
0,0-12,4283,5.02
1,12-19,2724,5.74
2,20-25,4773,4.85
3,25-29,6986,5.11
4,30-34,8860,5.66
5,35-39,10032,6.13
6,40-45,11562,6.76
7,45-49,14011,7.10
8,50-54,19064,7.33
9,55-59,23384,7.67


In [9]:
spark.sql(f"""

        select count(h.personId) as count_of_non_vaccinated_patients
               , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_nonVac}

              and length(h.dischargeDate) > 3
              and length(h.serviceDate) > 3


              and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
                    or v.vaccine_date is not null)  

""").toPandas()

,count_of_non_vaccinated_patients,mean_length_of_stay
0,8092,7.15


In [10]:
spark.sql(f"""

          select
              case
                when age >= 90 then "90+"
                when age >= 85 and age < 90 then "85-89"
                when age >= 80 and age < 85 then "80-85"
                when age >= 75 and age < 80 then "75-79"
                when age >= 70 and age < 75 then "70-74"
                when age >= 65 and age < 70 then "65-69"
                when age >= 60 and age < 65 then "60-65"
                when age >= 55 and age < 60 then "55-59"
                when age >= 50 and age < 55 then "50-54"
                when age >= 45 and age < 50 then "45-49"
                when age >= 40 and age < 45 then "40-45"
                when age >= 35 and age < 40 then "35-39"
                when age >= 30 and age < 35 then "30-34"
                when age >= 25 and age < 30 then "25-29"
                when age >= 20 and age < 25 then "20-25"
                when age >= 12 and age < 20 then "12-19"
                when age >= 0 and age < 12 then "0-12"
                else "unknown"
                end as age_range
              , count(h.personId) as count_of_vaccinated_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
                    and v.vaccine_date is not null)  
               
        group by 1
        order by 1
     
""").toPandas()

,age_range,count_of_vaccinated_patients,mean_length_of_stay
0,12-19,1,4.00
1,20-25,4,5.25
2,25-29,1,2.00
3,30-34,9,3.67
4,35-39,5,4.60
5,40-45,13,4.08
6,45-49,12,3.83
7,50-54,14,5.71
8,55-59,25,4.52
9,60-65,17,7.94


In [11]:
spark.sql(f"""

          select count(h.personId) as count_of_vaccinated_patients
                , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and length(h.dischargeDate) > 3
              and length(h.serviceDate) > 3


              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
                    and v.vaccine_date is not null)  
               
      
""").toPandas()

,count_of_vaccinated_patients,mean_length_of_stay
0,339,5.28


In [12]:
spark.sql(f"""

          select
              case
                when age >= 90 then "90+"
                when age >= 85 and age < 90 then "85-89"
                when age >= 80 and age < 85 then "80-85"
                when age >= 75 and age < 80 then "75-79"
                when age >= 70 and age < 75 then "70-74"
                when age >= 65 and age < 70 then "65-69"
                when age >= 60 and age < 65 then "60-65"
                when age >= 55 and age < 60 then "55-59"
                when age >= 50 and age < 55 then "50-54"
                when age >= 45 and age < 50 then "45-49"
                when age >= 40 and age < 45 then "40-45"
                when age >= 35 and age < 40 then "35-39"
                when age >= 30 and age < 35 then "30-34"
                when age >= 25 and age < 30 then "25-29"
                when age >= 20 and age < 25 then "20-25"
                when age >= 12 and age < 20 then "12-19"
                when age >= 0 and age < 12 then "0-12"
                else "unknown"
                end as age_range
              , count(h.personId) as count_of_all_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue}

        group by 1
        order by 1
     
""").toPandas()

,age_range,count_of_all_patients,mean_length_of_stay
0,0-12,4283,5.02
1,12-19,2725,5.74
2,20-25,4777,4.85
3,25-29,6987,5.11
4,30-34,8869,5.66
5,35-39,10037,6.13
6,40-45,11577,6.76
7,45-49,14023,7.09
8,50-54,19078,7.33
9,55-59,23411,7.66


In [13]:
spark.sql(f"""

        select count(h.personId) as count_of_all_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue}

     
""").toPandas()

,count_of_all_patients,mean_length_of_stay
0,293715,7.74
